# HydroSOS Publish Status Products
#### Jose Valles (jose.valles.leon@gmail.com)

In [1]:
import os
import requests
import json 

# This code line allows to remove warning produce by verify = False
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

## Send the results to Dinagua Web Service for publishing

After creating the json file for the corresponding year-month, we will update the Web Service using the following process

First, we are going to obtain the token

In [3]:
# Ambiente produccion
url_base = 'https://www.ambiente.gub.uy/dinaguaws/'
# Username and password
user_dinagua = {"user":"FEWS-Uruguay","password":"4MEzOeKFgp0pj9lATSIF"}
# Get the token from DINAGUA WS
response_token = requests.post(url=url_base + "gettoken", json=user_dinagua, verify=False);
response_token.close()
# Extract token and assign it to a variable
token = response_token.text
# Add the token to a bearer authorization
headers = {"Authorization" : "Bearer " + token}

Define the parameters for publishing the status results

In [103]:
# Locate the corresponding json file
json_file = '../waterbalance/output_json/01_month/2024-04.json'
date_name = json_file.split('/')[-1].split('.')[0]
# scale: 1, 3, 12
cat_scale = 1
# ts: Estaciones or Modelos
ts = "Modelos"

POST Method for creating the filter Id

In [104]:
# create the json dictonary
dataFiltro = {"fecha": date_name,"temporalidad": cat_scale,"serietemporal": ts}

# POST Save Filter
post_filtro = requests.post(url=url_base + "estadohidro/datoscat/guardarFiltro", headers = headers, json=dataFiltro, verify=False)
post_filtro.close()
print(post_filtro.text)

Filtro guardado con éxito


GET Method for extracting the FilterID

In [105]:
# Create the json dictonary
params_filtros = {"fecha" : date_name, "temporalidad" : cat_scale, "serietemporal" : ts}
# Send requests
get_filtro_id = requests.get(url = url_base + "estadohidro/filtros",headers=headers, params=params_filtros, verify=False)
get_filtro_id.close()
# print id
id = json.loads(get_filtro_id.content.decode('utf-8'))
id = id[0]["id"]
print(id)

26


POST HydroSOS Status for the corresponding Month/Year

In [106]:
# Creating the Request Body
datosCat = {"filtrosId": {id}}
# importing the json file from the output directory
json_body = open(json_file)
json_body = json.load(json_body)
# POST HydroSOS status 
post_estado = requests.post(url=url_base+"estadohidro/datoscat/guardarDatosCat",headers=headers,params=datosCat,json=json_body,verify=False)
post_estado.close()
print(post_estado.text)

DatosCat guardado con éxito


We are going to see if the hydrological status is correctly added in the service

In [107]:
params_status = {"serietemporal": ts, "fecha": date_name, "temporalidad": cat_scale}
get_status = requests.get(url=url_base+"estadohidro/datoscat",headers=headers,params=params_status,verify=False)
get_status.close()
# Convert it to json and dataframe
response_status = json.loads(get_status.content.decode('utf-8'))
response_status

[{'id': 671, 'category': 5, 'stationID': '22', 'filtros_id': 26},
 {'id': 672, 'category': 5, 'stationID': '57', 'filtros_id': 26},
 {'id': 673, 'category': 4, 'stationID': '41', 'filtros_id': 26},
 {'id': 674, 'category': 5, 'stationID': '53', 'filtros_id': 26},
 {'id': 675, 'category': 5, 'stationID': '18', 'filtros_id': 26},
 {'id': 676, 'category': 5, 'stationID': '29', 'filtros_id': 26},
 {'id': 677, 'category': 4, 'stationID': '40', 'filtros_id': 26},
 {'id': 678, 'category': 4, 'stationID': '43', 'filtros_id': 26},
 {'id': 679, 'category': 5, 'stationID': '67', 'filtros_id': 26},
 {'id': 680, 'category': 5, 'stationID': '55', 'filtros_id': 26},
 {'id': 681, 'category': 4, 'stationID': '45', 'filtros_id': 26},
 {'id': 682, 'category': 5, 'stationID': '60', 'filtros_id': 26},
 {'id': 683, 'category': 4, 'stationID': '50', 'filtros_id': 26},
 {'id': 684, 'category': 4, 'stationID': '33', 'filtros_id': 26},
 {'id': 685, 'category': 5, 'stationID': '27', 'filtros_id': 26},
 {'id': 68